In [ ]:
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

def preprocess_data(image_folder, annotation_file):
    with open(annotation_file, 'r') as f:
        annotations = json.load(f)
    
    images = []
    car_colors = []
    car_counts = []
    people_counts = []

    # Gather all unique car colors
    all_colors = set()
    for key, value in annotations.items():
        for region in value['regions']:
            attributes = region['region_attributes']
            colors = attributes['car color'].split('\n')
            all_colors.update(colors)
    
    # Initialize encoder with all unique colors
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoder.fit(np.array(list(all_colors)).reshape(-1, 1))

    for key, value in annotations.items():
        img_path = f"{image_folder}/{value['filename']}"
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to read image: {img_path}")
            continue
        
        h, w, _ = img.shape

        for region in value['regions']:
            shape = region['shape_attributes']
            attributes = region['region_attributes']
            
            x, y, width, height = shape['x'], shape['y'], shape['width'], shape['height']
            roi = img[y:y+height, x:x+width]

            images.append(cv2.resize(roi, (128, 128)))

            colors = attributes['car color'].split('\n')
            try:
                car_colors1 = encoder.transform(np.array(colors).reshape(-1, 1))
                if car_colors1.shape[0] == 0:
                    print(f"No valid color data for: {colors}")
                    continue
                car_colors.append(car_colors1.mean(axis=0))  # Aggregate to a single vector
            except Exception as e:
                print(f"Error encoding colors: {colors}. Error: {e}")

            car_counts.append(int(attributes['car count']))

            people_count = attributes['people count'].split('\n')
            males = int(people_count[0].split(':')[1])
            females = int(people_count[1].split(':')[1])
            people_counts.append({'males': males, 'females': females})

    images = np.array(images) / 255.0  # Normalize images
    car_colors_one_hot = np.array(car_colors)
    car_counts = np.array(car_counts)
    people_counts = np.array([list(d.values()) for d in people_counts])

    # Debug print statements
    print(f"Images: {len(images)}")
    print(f"Car Colors: {len(car_colors_one_hot)}")
    print(f"Car Counts: {len(car_counts)}")
    print(f"People Counts: {len(people_counts)}")
    
    # Ensure all arrays have the same length
    if len(images) != len(car_colors_one_hot) or len(images) != len(car_counts) or len(images) != len(people_counts):
        raise ValueError("Mismatch in the number of samples between arrays")

    return train_test_split(images, car_colors_one_hot, car_counts, people_counts, test_size=0.2, random_state=42)

# Path to images folder and annotation file
image_folder = r'C:\Users\chais\Downloads\IMAGES'
annotation_file = r'C:\Users\chais\Downloads\via_region_data (1).json'

# Prepare data
X_train, X_test, y_color_train, y_color_test, y_count_train, y_count_test, y_people_train, y_people_test = preprocess_data(image_folder, annotation_file)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout

def create_model():
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
    x = base_model.output
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)  # Dropout for regularization

    # Output layers
    color_output = tf.keras.layers.Dense(9, activation='softmax', name='color_output')(x)
    count_output = tf.keras.layers.Dense(1, activation='linear', name='count_output')(x)
    people_output = tf.keras.layers.Dense(2, activation='linear', name='people_output')(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=[color_output, count_output, people_output])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss={'color_output': 'categorical_crossentropy', 'count_output': 'mse', 'people_output': 'mse'},
                  metrics={'color_output': 'accuracy', 'count_output': 'mae', 'people_output': 'mae'})
    return model


In [ ]:
# Define the model
def create_model():
    input_img = tf.keras.layers.Input(shape=(128, 128, 3))
    
    # Shared layers
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input_img)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)

    # Output layers
    color_output = tf.keras.layers.Dense(9, activation='softmax', name='color_output')(x)
    count_output = tf.keras.layers.Dense(1, activation='linear', name='count_output')(x)
    people_output = tf.keras.layers.Dense(2, activation='linear', name='people_output')(x)

    model = tf.keras.models.Model(inputs=input_img, outputs=[color_output, count_output, people_output])
    model.compile(optimizer='adam', 
                  loss={'color_output': 'categorical_crossentropy', 'count_output': 'mse', 'people_output': 'mse'},
                  metrics={'color_output': 'accuracy', 'count_output': 'mae', 'people_output': 'mae'})
    return model



In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout

def create_model():
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
    x = base_model.output
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)  # Dropout for regularization

    # Output layers
    color_output = tf.keras.layers.Dense(9, activation='softmax', name='color_output')(x)
    count_output = tf.keras.layers.Dense(1, activation='linear', name='count_output')(x)
    people_output = tf.keras.layers.Dense(2, activation='linear', name='people_output')(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=[color_output, count_output, people_output])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss={'color_output': 'categorical_crossentropy', 'count_output': 'mse', 'people_output': 'mse'},
                  metrics={'color_output': 'accuracy', 'count_output': 'mae', 'people_output': 'mae'})
    return model


In [ ]:
# Create and train the model
model = create_model()
model.fit(X_train, {'color_output': y_color_train, 'count_output': y_count_train, 'people_output': y_people_train},
          validation_data=(X_test, {'color_output': y_color_test, 'count_output': y_count_test, 'people_output': y_people_test}),
          epochs=10, batch_size=16)

In [ ]:
model.save('my_model.h5')
